<a href="https://colab.research.google.com/github/AK-Github-0/LangChain/blob/main/Building_a_Query_Analysis_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [28]:
# %pip install -qU langchain langchain-community langchain-openai youtube-transcript-api pytube langchain-chroma

In [29]:
pip install langchain-cohere

In [30]:
pip install youtube-transcript-api

In [31]:
pip install langchain-community

In [32]:
pip install pytube

In [33]:
pip install langchain-chroma

In [34]:
pip install chroma-hnswlib onnxruntime opentelemetry-api opentelemetry-exporter-otlp-proto-grpc opentelemetry-instrumentation-fastapi opentelemetry-sdk importlib-resources bcrypt kubernetes mmh3 coloredlogs deprecated importlib-metadata requests-oauthlib oauthlib opentelemetry-exporter-otlp-proto-common opentelemetry-proto opentelemetry-instrumentation-asgi opentelemetry-instrumentation opentelemetry-semantic-conventions opentelemetry-util-http asgiref monotonic backoff


In [35]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r")

··········


# Load documents

In [36]:
from langchain_community.document_loaders import YoutubeLoader

urls = [
    "https://www.youtube.com/watch?v=HAn9vnJy6S4",
    "https://www.youtube.com/watch?v=dA1cHGACXCo",
    "https://www.youtube.com/watch?v=ZcEMLz27sL4",
    "https://www.youtube.com/watch?v=hvAPnpSfSGo",
    "https://www.youtube.com/watch?v=EhlPDL4QrWY",
    "https://www.youtube.com/watch?v=mmBo8nlu2j0",
    "https://www.youtube.com/watch?v=rQdibOsL1ps",
    "https://www.youtube.com/watch?v=28lC4fqukoc",
    "https://www.youtube.com/watch?v=es-9MgxB-uc",
    "https://www.youtube.com/watch?v=wLRHwKuKvOE",
    "https://www.youtube.com/watch?v=ObIltMaRJvY",
    "https://www.youtube.com/watch?v=DjuXACWYkkU",
    "https://www.youtube.com/watch?v=o7C9ld6Ln-M",
]
docs = []
for url in urls:
    docs.extend(YoutubeLoader.from_youtube_url(url, add_video_info=True).load())

In [37]:
import datetime

# Add some additional metadata: what year the video was published
for doc in docs:
    doc.metadata["publish_year"] = int(
        datetime.datetime.strptime(
            doc.metadata["publish_date"], "%Y-%m-%d %H:%M:%S"
        ).strftime("%Y")
    )

In [38]:
[doc.metadata["title"] for doc in docs]

['OpenGPTs',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangGraph: Multi-Agent Workflows',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Auto-Prompt Builder (with Hosted LangServe)',
 'Build a Full Stack RAG App With TypeScript',
 'Getting Started with Multi-Modal LLMs',
 'SQL Research Assistant',
 'Skeleton-of-Thought: Building a New Template from Scratch',
 'Benchmarking RAG over LangChain Docs',
 'Building a Research Assistant from Scratch',
 'LangServe and LangChain Templates Webinar']

In [39]:
docs[0].metadata

{'source': 'HAn9vnJy6S4',
 'title': 'OpenGPTs',
 'description': 'Unknown',
 'view_count': 8785,
 'thumbnail_url': 'https://i.ytimg.com/vi/HAn9vnJy6S4/hq720.jpg',
 'publish_date': '2024-01-31 00:00:00',
 'length': 1530,
 'author': 'LangChain',
 'publish_year': 2024}

In [40]:
docs[0].page_content[:500]

"hello today I want to talk about open gpts open gpts is a project that we built here at linkchain uh that replicates the GPT store in a few ways so it creates uh end user-facing friendly interface to create different Bots and these Bots can have access to different tools and they can uh be given files to retrieve things over and basically it's a way to create a variety of bots and expose the configuration of these Bots to end users it's all open source um it can be used with open AI it can be us"

# Indexing documents

In [41]:
from langchain_chroma import Chroma
from langchain_cohere import CohereEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
chunked_docs = text_splitter.split_documents(docs)
embeddings = CohereEmbeddings()
vectorstore = Chroma.from_documents(
    chunked_docs,
    embeddings,
)

# Retrieval without query analysis

In [42]:
search_results = vectorstore.similarity_search("how do I build a RAG agent")
print(search_results[0].metadata["title"])
print(search_results[0].page_content[:500])

Streaming Events: Introducing a new `stream_events` method
with the agent executor so let's restart the kernel so we can see what's going on here if you haven't if you don't know what agents are check them out in a separate video I'll link to one in the description basically with agents the first thing we're going to do is we're going to define a model and we're going to make sure that streaming set equal to true this is so that it's this is necessary so that it it streams no matter where it's called from so in an agent it will be called within the agen


In [43]:
search_results = vectorstore.similarity_search("videos on RAG published in 2023")
print(search_results[0].metadata["title"])
print(search_results[0].metadata["publish_date"])
print(search_results[0].page_content[:500])

Getting Started with Multi-Modal LLMs
2023-12-20 00:00:00
but I saw this quite a bit in my recent on my recent runs uh it did not seem correlated to resolution it seems uh more or less random uh I think it's also skewed the prior results a bit because the scoring um the correct scoring um may have been affected by uh a number of the trials for a basically erroring app out and failing to produce a correct or incorrect score so it injected some noise into the prior result unfortunately um so anyway this just a caveat if you're think about using this in p


# Query analysis

## Query schema

In [44]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    query: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    publish_year: Optional[int] = Field(None, description="Year video was published")

## Query generation

In [45]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a list of database queries optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatCohere(model="command-r")
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

In [46]:
query_analyzer.invoke("how do I build a RAG agent")

Search(query='build RAG agent', publish_year=None)

In [47]:
query_analyzer.invoke("videos on RAG published in 2023")

Search(query='RAG', publish_year=2023)

# Retrieval with query analysis

In [51]:
from typing import List

from langchain_core.documents import Document

In [52]:
def retrieval(search: Search) -> List[Document]:
    if search.publish_year is not None:
        # This is syntax specific to Chroma,
        # the vector database we are using.
        _filter = {"publish_year": {"$eq": search.publish_year}}
    else:
        _filter = None
    return vectorstore.similarity_search(search.query, filter=_filter)

In [53]:
retrieval_chain = query_analyzer | retrieval

In [54]:
results = retrieval_chain.invoke("RAG tutorial published in 2023")

In [55]:
[(doc.metadata["title"], doc.metadata["publish_date"]) for doc in results]

[('SQL Research Assistant', '2023-12-19 00:00:00'),
 ('SQL Research Assistant', '2023-12-19 00:00:00'),
 ('Getting Started with Multi-Modal LLMs', '2023-12-20 00:00:00'),
 ('Getting Started with Multi-Modal LLMs', '2023-12-20 00:00:00')]